In [4]:
train_FD001=pd.read_csv("C:/Users/D-InfoTech/Desktop/Jupyter project/train_FD001.txt", sep=' ', header=None)

In [2]:
#import librabries

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
#Load the Training Data

In [3]:
column_names = [
    'engine_id', 'cycle',
    'op_setting_1', 'op_setting_2', 'op_setting_3',
] + [f'sensor_{i}' for i in range(1, 22)]

df = pd.read_csv('C:/Users/D-InfoTech/Desktop/Jupyter project/train_FD001.txt', sep=' ', header=None)
df.dropna(axis=1, inplace=True)  # Drop empty columns at the end
df.columns = column_names


In [7]:
#Calculate RUL (Remaining Useful Life)

In [8]:
# Get max cycle per engine
rul_df = df.groupby('engine_id')['cycle'].max().reset_index()
rul_df.columns = ['engine_id', 'max_cycle']

# Merge with main data to calculate RUL
df = df.merge(rul_df, on='engine_id')
df['RUL'] = df['max_cycle'] - df['cycle']
df.drop('max_cycle', axis=1, inplace=True)

In [16]:
import sympy as sp
import numpy as np

# Step 1: Define symbolic variables
x, y, z = sp.symbols('x y z')     # position variables
vx, vy, vz = sp.symbols('vx vy vz')  # velocity variables
phi, theta, psi = sp.symbols('phi theta psi')       # angle variable
g = sp.Symbol('g')                # gravity

# Step 2: Define your function
# Example: acceleration a_x = -vz*y + g*sin(theta)
f_x = (math.cos(theta)*math.cos(psi))*vx + (math.cos(theta)*math.sin(psi))*vy + (-math.sin(theta))*vz
f_y = (math.sin(phi)*math.sin(theta)*math.cos(psi)-math.cos(phi)*math.sin(psi))*vx + 

# Step 3: Compute partial derivatives
# ∂a_x/∂y
partial_y = sp.diff(a_x, y)

# ∂a_x/∂theta
partial_theta = sp.diff(a_x, theta)

# Step 4: Print them
print("∂a_x/∂y =", partial_y)
print("∂a_x/∂theta =", partial_theta)

# Optional: Evaluate numerically
subs = {
    y: 2.0,
    vz: 3.0,
    theta: 0.5,
    g: 9.81
}
print("∂a_x/∂y (value) =", partial_y.evalf(subs=subs))
print("∂a_x/∂theta (value) =", partial_theta.evalf(subs=subs))


SyntaxError: invalid syntax (985248743.py, line 13)

In [17]:
import numpy as np
import sympy as sp
import math
x = sp.symbols('x')
y = sp.symbols('y')
z = sp.symbols('z')
U = sp.symbols('U')
V = sp.symbols('V')
W = sp.symbols('W')
phi = sp.symbols('phi')
theta = sp.symbols('theta')
psi = sp.symbols('psi')
cphi, sphi = math.cos(phi), math.sin(phi)
ctheta, stheta = math.cos(theta), math.sin(theta)
cpsi, spsi = math.cos(psi), math.sin(psi)

C_bn = np.array([
    [ctheta * cpsi, ctheta * spsi, -stheta],
    [sphi * stheta * cpsi - cphi * spsi, sphi * stheta * spsi + cphi * cpsi, sphi * ctheta],
    [cphi * stheta * cpsi + sphi * spsi, cphi * stheta * spsi - sphi * cpsi, cphi * ctheta]
])

# World velocity from body
pos_dot = C_bn.T @ np.array([[U], [V], [W]])

ModuleNotFoundError: No module named 'sympy'

In [6]:
pos_dot

array([[6.25751774],
       [4.48595099],
       [4.20947924]])

In [14]:
import sympy as sp
from sympy import symbols, sin, cos, tan, sec, Matrix

ModuleNotFoundError: No module named 'sympy'

In [18]:
import sympy
from sympy import symbols, sin, cos, tan, sec, Matrix

# 1. Define the symbolic state variables
x, y, z = symbols('x y z')
U, V, W = symbols('U V W') # Body frame velocities
phi, theta, psi = symbols('phi theta psi') # Euler angles (roll, pitch, yaw)

# Define symbolic inputs/controls (p, q, r are angular velocities in body frame)
# ax, ay, az are accelerations in body frame (e.g., from external forces/motors)
p, q, r = symbols('p q r') # Angular velocities in body frame
ax, ay, az = symbols('ax ay az') # Body frame accelerations (control inputs or external forces)
g = symbols('g') # Gravity constant

# The full state vector 'X' for differentiation
state_vector = Matrix([x, y, z, U, V, W, phi, theta, psi])

# Define the C_bn (or rotation matrix from body to navigation frame) components
# c(theta) = cos(theta), s(theta) = sin(theta) etc.
c_phi, s_phi = cos(phi), sin(phi)
c_theta, s_theta = cos(theta), sin(theta)
c_psi, s_psi = cos(psi), sin(psi)

# C_bn Matrix (from the image)
C_bn = Matrix([
    [c_theta * c_psi, c_theta * s_psi, -s_theta],
    [s_phi * s_theta * c_psi - c_phi * s_psi, s_phi * s_theta * s_psi + c_phi * c_psi, s_phi * c_theta],
    [c_phi * s_theta * c_psi + s_phi * s_psi, c_phi * s_theta * s_psi - s_phi * c_psi, c_phi * c_theta]
])

# Transpose of C_bn
C_bn_T = C_bn.T

# Linear velocities in the navigation frame (rows 1-3 of f(x,u))
# This is C_bn_T * [U, V, W]^T
# Note: In standard robotics, often this is C_nb * [U, V, W]^T, where C_nb = C_bn.T
# Based on your image, it is indeed C_bn.T * [U, V, W]^T
linear_vel_nav_frame = C_bn_T * Matrix([U, V, W])
x_dot = linear_vel_nav_frame[0]
y_dot = linear_vel_nav_frame[1]
z_dot = linear_vel_nav_frame[2]


# Angular velocities (rows 7-9 of f(x,u)) - Euler angle derivatives
# This part converts body angular velocities (p,q,r) to Euler angle rates (phi_dot, theta_dot, psi_dot)
# Note: tan(theta) can be problematic near theta = +/- pi/2 (gimbal lock).
phi_dot = p + q * s_phi * tan(theta) + r * c_phi * tan(theta)
theta_dot = q * c_phi - r * s_phi
psi_dot = q * s_phi * sec(theta) + r * c_phi * sec(theta)


# Accelerations in the body frame (rows 4-6 of f(x,u))
# These are typically derived from Newton's laws and gravitational effects
# This part seems to be the time derivative of U, V, W (body frame accelerations)
U_dot = ax + V*r - W*q + g*s_theta
V_dot = ay - U*r + W*p - g*c_theta*s_phi # Typo in image? Should be s(phi) or c(phi)? Assuming s(phi)
                                        # Image shows gc(phi) but typically it's related to pitch and roll
                                        # If 'c(phi)' in image is cos(phi), this looks like a gravity component
                                        # Let's assume it's gc(phi) as written in image to match
V_dot = ay - U*r + W*p - g*c_phi # Using gc(phi) from image for V_dot's gravity term
W_dot = az + U*q - V*p - g*c_theta*c_phi # Using gc(theta) from image for W_dot's gravity term


# Construct the full f(x,u) vector function
f_sym = Matrix([
    x_dot,
    y_dot,
    z_dot,
    U_dot,
    V_dot,
    W_dot,
    phi_dot,
    theta_dot,
    psi_dot
])

print("Symbolic function f(x,u):")
# for i, expr in enumerate(f_sym):
#     print(f"f_{i+1} = {expr}")
print(f_sym)

print("\n-------------------------------------------------\n")

# 2. Compute the Jacobian Matrix (partial derivatives of f_sym with respect to state_vector)
# This is df/dx_i for each f_j component
Jacobian_matrix = f_sym.jacobian(state_vector)

print("Jacobian Matrix (partial derivatives of f(x,u) with respect to x, y, z, U, V, W, phi, theta, psi):")
print(Jacobian_matrix)

ModuleNotFoundError: No module named 'sympy'